# Assignment 4: Encoding models (10 pts in total)

In [1]:
# Install torchextractor for facilitating feature extraction
!pip install torchextractor

# Import packages
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter('ignore')

import torch
from torchvision import models, transforms
import torchextractor as tx
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold

In [2]:
# Download the dataset
!gdown 1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS
# Extract the dataset and remove the tar file
!mkdir -p natural_scenes_demo && tar -xzf natural_scenes_demo.tar.gz -C natural_scenes_demo
!rm natural_scenes_demo.tar.gz

Downloading...
From (original): https://drive.google.com/uc?id=1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS
From (redirected): https://drive.google.com/uc?id=1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS&confirm=t&uuid=ecc4bd62-10c1-4584-a8c5-6fa3ef1dfe1e
To: /content/natural_scenes_demo.tar.gz
100% 395M/395M [00:03<00:00, 109MB/s]
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'SCHILY.fflags'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown exte

### Dataset description
This dataset contains only one subject, who viewed 1000 images in the fMRI scanner. The images are naturalistic images. The fMRI data has been preprocessed, and voxel-wise BOLD responses has been estimated. The voxel-wise BOLD responses are stored in the file 'response_data.parquet', with rows representing voxels and columns representing images. The file 'metadata.csv' records detailed information on each voxel.

The file 'stimulus_data.csv' records detailed information on each image. The image files are located at the folder 'stimulus_set'.

In this assignment, you will fit an encoding model with AlexNet activations to BOLD responses in the region PPA.

The BOLD responses in PPA have been extracted for you with the code below.

In [ ]:
# Load voxel metadata
df = pd.read_csv('natural_scenes_demo/metadata.csv')

# Select voxels in PPA with reliability > 0.1
voxel_ids = df.loc[
    (df['roi_name'] == 'PPA') & (df['voxel_reliability'] > 0.1),
    'voxel_id'
].to_numpy()

# Load voxel-wise BOLD responses
data = pd.read_parquet('natural_scenes_demo/response_data.parquet')
data = data.set_index('voxel_id')
bold_responses = data.loc[voxel_ids].to_numpy()  # shape: (num_voxels, num_images)
print(f'Number of voxels in PPA with reliability > 0.1: {len(voxel_ids)}')
print(f'BOLD responses shape: {bold_responses.shape}')

#### ✏️ Do it yourself (0.5 pts):
How is reliablity of BOLD responses generally calculated? Why do we want to remove voxels with low reliability?

Write your answer here:
> **Answer:**
> The reliability of BOLD responses is calculated with correlation coefficient R^2.
>   


#### ✏️ Do it yourself (2 pt):
Exact layer activations of 1000 images from the last convolutional layer (after ReLU) in Alexnet.
_Hint: Make sure the layer you want is properly indexed_

In [ ]:
# Insert your code here


# flatten the activations into vectors
activations_conv5_flat = activations_conv5.reshape(len(activations_conv5), -1)

#### ✏️ Do it yourself (0.5 pt):
You should have extracted 43264 features from each image in the data set from conv5. Now randomly split 80% of the data as the training set and 20% as the testing set. \
_Hint: Use `train_test_split` from sklearn. Set random_state=42 so the split are reproducible_

In [ ]:
# Insert your code here

X_train, X_test, y_train, y_test = ...

#### ✏️ Do it yourself (1 pt):
Now you want to do dimension reduction by performing PCA on the 43264 features and reduce it to 50 dimensions. \

Why do you want to do dimension reduction? Should you perform PCA only on the training dataset or on the whole dataset, and why?

Write your answer here:
> **Answer:**
>
>   

#### ✏️ Do it yourself (1 pt):
Perform PCA to reduce the dimension to 50.

In [ ]:
# Insert your code here



#### ✏️ Do it yourself (1 pt):
Build encoding models by conducting voxel-wise Ridge regression and predicting on the test dataset. \
_Hint: Use `Ridge` from sklearn. Set the alpha = 1_

In [ ]:
# insert your code below

# ridge regression model
ridge_model = Ridge(alpha=1.0)

# fit the model
...



# predict on the test set
y_pred = ...


#### ✏️ Do it yourself (1 pt):
Evaluate the performance of your encoding model by computing the correlation coefficient. Get a summary score for the ROI (i.e., PPA) by calculating the _median_ of all voxels. \
_Hint: Use np.corrcoef to compute correlation_

In [ ]:
# Insert your code below

# compute r for each voxel
r_scores = ...

# compute the median r across voxels
median_r = ...

#### ✏️ Do it yourself (3 pt):
Now you will wrap up all those steps above into one code cell, and put the encoding models under a _two-folds_ cross-validation scheme. Note that to get the final performance evaluation, concatenate each fold into one big data and compute the correlation.Output the median r of all voxels.\
_Hint: Use `KFold` to split folds. Remember to set random_state=42 for reproducibility_

In [ ]:
# Insert your code here

